In [3]:
# Merging sales data with Census income

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline

In [5]:
income_df = pd.read_csv('~/Box Sync/CDS/Developer Advocacy/Open Data/uscensus/x19_income.csv.gz', 
                        usecols=['GEOID','B19001e1','B19001e2','B19001e3','B19001e4','B19001e5','B19001e6','B19001e7','B19001e8','B19001e9','B19001e10','B19001e11','B19001e12','B19001e13','B19001e14','B19001e15','B19001e16','B19001e17',
                                 'B19019e1','B19019e2','B19019e3','B19019e4','B19019e5','B19019e6','B19019e7','B19019e8',
                                 'B19049e1','B19049e2','B19049e3','B19049e4','B19049e5'])
income_df.columns=['GEOID','hhinctotalhh','hhinclt10k','hhinc10-15k','hhinc15-20k','hhinc20-25k','hhinc25-30k','hhinc30-35k','hhinc35-40k','hhinc40-45k','hhinc45-50k','hhinc50-60k','hhinc60-75k','hhinc75-100k','hhinc100-125k','hhinc125-150k','hhinc150-200k','hhincgt200k','medhhincbysizetot','medhhincbysize1p','medhhincbysize2p','medhhincbysize3p','medhhincbysize4p','medhhincbysize5p','medhhincbysize6p','medhhincbysize7p','medhhincbyagetotal','medhhincbyagelt25','medhhincbyage25-44','medhhincbyage45-64','medhhincbyagegt65']
## NEED TO DEAL WITH ISSUE WHERE DATA IS SUPPRESSED BY REMOVING ROWS WITH ZEROS
### SUCH AS GEOID 86000US01003 (Amherst, MA)
income_df = income_df.set_index('GEOID')
income_df.head()

,hhinctotalhh,hhinclt10k,hhinc10-15k,hhinc15-20k,hhinc20-25k,hhinc25-30k,hhinc30-35k,hhinc35-40k,hhinc40-45k,hhinc45-50k,...,medhhincbysize3p,medhhincbysize4p,medhhincbysize5p,medhhincbysize6p,medhhincbysize7p,medhhincbyagetotal,medhhincbyagelt25,medhhincbyage25-44,medhhincbyage45-64,medhhincbyagegt65
GEOID,,,,,,,,,,,,,,,,,,,,,
86000US01001,7056,494,259,355,341,298,325,335,210,381,...,75128,83480,99549,128295,96413,58733,37533,68766,79127,29383
86000US01002,9248,912,707,570,352,543,377,335,234,270,...,59777,53688,92813,34306,197143,54422,21094,48378,102500,69452
86000US01003,16,0,0,0,0,0,0,0,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86000US01005,1937,61,42,88,113,99,88,40,19,24,...,83274,93500,83750,NaN,193750,68644,NaN,98000,76667,30265
86000US01007,5798,251,190,371,213,399,183,180,154,257,...,102337,105000,74788,98657,NaN,71875,21016,71976,89867,36173


In [6]:
# GET SALES DATA
sales_csv = "/Users/rajrsingh/Box Sync/CDS/Developer Advocacy/Open Data/sales2009/SalesJan2009.csv"
sales_df = pd.read_csv(sales_csv, dtype={"US Zip": np.str}, usecols=['Product','Price','US Zip','Country'])
# Just get US sales for a single product and get rid of the bad data where price is '13,000'
sales_df = sales_df[ (sales_df.Product == 'Product1') 
                      & (sales_df.Country == 'United States')
                      & (sales_df.Price != '13,000')]
sales_df.drop(['Product','Country'],inplace=True,axis=1)
sales_df['Price'] = pd.to_numeric(sales_df['Price'])
#sales_df['SALE'] = 1
sales_df['GEOID'] = "86000US" + sales_df['US Zip']
#sales_df.sort_values('Price',ascending=False)

# Simplify the data down to number of sales per zip
sales_df.drop(['Price','US Zip'],inplace=True,axis=1)
#sales_census = sales_df['GEOID'].value_counts().to_frame('Sales')
#sales_census['GEOID'] = sales_census.index
sales_counts = sales_df['GEOID'].value_counts()
sales_counts.name = 'Sales'
sales_counts.head(4)

86000US10007    9
86000US77002    7
86000US98104    5
86000US60608    4
Name: Sales, dtype: int64

In [13]:
# Merge sales to census

cs = income_df.join(sales_counts, how='right')
cs.loc[:,'medhhincbyagetotal':'medhhincbyagegt65'].describe()

,medhhincbyagetotal,medhhincbyagelt25,medhhincbyage25-44,medhhincbyage45-64,medhhincbyagegt65
count,304.000000,252.000000,304.000000,304.000000,303.000000
mean,76178.167763,35239.150794,82372.003289,89658.562500,50312.650165
std,36094.736528,20312.395534,39541.968909,43695.095669,24343.014565
min,16069.000000,2499.000000,23387.000000,12819.000000,9739.000000
25%,50307.000000,21714.000000,53302.500000,59379.250000,34128.000000
50%,69855.500000,31553.500000,75870.000000,82258.000000,46140.000000
75%,95624.500000,45303.750000,101802.250000,114105.000000,61451.000000
max,216037.000000,145556.000000,250001.000000,250001.000000,172143.000000


In [14]:
income_df.loc[:,'medhhincbyagetotal':'medhhincbyagegt65'].describe()

,medhhincbyagetotal,medhhincbyagelt25,medhhincbyage25-44,medhhincbyage45-64,medhhincbyagegt65
count,31859.000000,17468.000000,29339.000000,30836.000000,30124.000000
mean,52112.585863,31938.903652,58650.997035,61904.390809,37330.144436
std,22333.423430,18179.070134,26073.017267,27463.393038,16499.543691
min,2499.000000,2499.000000,2499.000000,2499.000000,2499.000000
25%,38272.500000,20053.250000,42122.500000,44434.250000,27336.750000
50%,47778.000000,28986.500000,54221.000000,57188.000000,34167.000000
75%,60691.000000,40236.000000,69636.500000,73376.250000,43203.000000
max,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000
